## Concha

Concha calculcates the optimal number of things to make/order of perishables.
It uses machine learning, the day of the week, the weather, and the profit margin
of each product to optimally balance filling customer demand and reducing waste.

Open this notebook up in Google Colab to run it on Google's computers (it's free...and amazing).
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ready4theCrush/concha/blob/master/notebooks/concha_examples.ipynb)

In [ ]:
# Run this cell first to bring in the concha code
!pip -q install -i https://test.pypi.org/simple/ concha
from concha import Planner

In [ ]:
# First, let's run a simulation for muffins.
# Let's say they're made in batches of 4, the marginal cost is
# $1.50 per muffin ($6 per batch), and they sell for $3.00 each.
sim_planner = Planner(
    planner_name="cafe_2",
    batch_size = 4,
    batch_cost= 6.0,
    unit_sale_price= 3.0
)

# We'll simulate 180 days of sales for 5 kinds of muffins
sim_planner.simulate_history(
    num_days=18,
    stockout_prob=0.5,
    demand_mean=100,
    demand_std=10,
    num_products = 5
) 

In [ ]:
# We'll compare profits from two different ways to plan production.
# The first is to use the average for weekdays and weekends from past
# days to plan future production. The other uses machine learning to
# maximize profit (revenue - waste). 
sim_planner.grid_search(
    param_grid={"model": [
        "MeanWeekPart",
        "ProfitMaximizer"
    ]},
    true_demand='demand'
)

In [ ]:
# To predict how much to make of each product, we
# just need to train the model for each product on the 
# transaction history first (which is simulated right now). 

sim_planner.train()
production = sim_planner.predict()

# The predictions are written to the forecast/forecast_production.csv file.
# Here's the first five rows of predictions.
production.head()

In [ ]:
# Importing the transaction history

# From the POS you need a csv dump of the transactions with
# three columns (in this order): timestamp, item name, quantity.
sim_planner.import_transactions()
sim_planner.update_settings()

In [ ]:
# Using the weather

# The weather history and forecasts come from NOAA. You'll
# need to get a NOAA api key from here: https://www.ncdc.noaa.gov/cdo-web/token
# You can set it in the planner with:
sim_planner.noaa_key("Yourkeyhere")

In [ ]:
# To find a station near your business location you can
# pick one off the NOAA map. Go here: https://www.ncdc.noaa.gov/cdo-web/search
#  - For "Select a Dataset" pick "Daily Summaries"
#  - For "Enter a Search Term" put in your city/state name. Hit "SEARCH" and a map should come up.
#  - On the left side of the map, above the search results and below the "Search" box,
#    click on "More Search Options" and check the "Air Temperature" and "Precipitation"
#    boxes. Then click "SEARCH" again. When you find the nearest station, click on it
#    and copy the ID. It will look like "GHCND:USC00448084". You can set it with:
sim_planner.noaa_station("GHCND:USC00448084")